## TPF
- Target Pixcel File
    - FITS format: Flexible Image Transport System
        - HDUs (header/data units)
            - Primary HDU followed by extensions
            - HDU
                - Header: `KEYNAME = value / comment string`
                - Data: Image pixels

        

## Kepler, K2, TESS
- Kepler recorded the brightness measurements at two different cadences: a Short Cadence (SC, 58.85 seconds) and a Long Cadence (LC, 29.4 minutes).
    - Kepler data are marked with quarters 1 - 4. This is because the telescope orientation needed to be reoriented every 3 months.
- K2 data are very similar to Kepler data, but are subject to higher levels of instrument noise due to the increased instability of the spacecraft.
- TESS covers a much larger area of the sky at a lower resolution. Compared to Kepler, TESS observes in several different cadence modes, including 20 seconds, 120 seconds, 10 minutes, and 30 minutes.
- Survey specific partition
    - Kepler quarter
    - K2 campaign
    - TESS sector
        - 4 cameras
        - Each camera has 4 CCDs
        - Each CCDs has 2048 x 2048 pixels (roughly 4M pixels)

## Terms
- BKJD: Barycentric Kepler Julian Date
- WCS: [World Coordinate System](https://fits.gsfc.nasa.gov/fits_wcs.html)
- CCD: Charge-Coupled Device
- Cadence: The interval durign which the CCD is actively exposed to light.
- FFI: TESS full frame image
- SAP: Simple aperture photometry
- PDCSAP: Pre-search Data Conditioning SAP
- FITS: Flexible Image Transport System
- PDC: Presearch Data Conditioning. PDC is responsible for preparing the raw photometric data (light curves) obtained from the Kepler spacecraft for further analysis by mitigating various systematic errors and noise sources.
    - Data Calibration
    - Systematic Error Correction
    - Outlier Removal
    - Discontinuity Correction
    - ...
- PDCMETHD: The light curve FITS header which PDC method was chosen
- MAST: Mikulski Archive for Space Telescopes. This is the archive that lightkurve searches. It inclues TESS, Kepler data for example.
- TOI: TESS object of interest
- TCE: Threshold crossing event
- ExoFOP: Exoplanet Follow-up Observing Program. Primarily used by researchers involved in the follow-up of exoplanet candidates to validate and characterize these planets.
- ExoMAST: Database for confirmed exoplanets.
- Contamination Ratio: The ratio of flux from nearby stars to the flux from the target star within the photometric aperture used by TESS. 0 means no contamination, 1 complete contamination.
- The Morgan-Keenan star classification system: OBAFGKM, Oh, Be a Fine Gal/Guy, Kiss Me.
- LBV: luminous blue variables (e.g, S Doradus) These star are not in the MK sequence.
- SSO: solar system objects. Lisk asteroids, comets, moons that cause big spike and dips in star light curves.
- The star radius cut-off of detecting transit exo-planet: 2~3 solar radii due to SNR.
    

## [tessLATTE](https://github.com/noraeisner/LATTE/tree/master?tab=readme-ov-file)
- May need to instal openssl@1.1 first before installing Python 3.7.8
    - `brew install openssl@1.1`
    - `brew reinstall openssl@1.1`
- Works with Python 3.7.8. Does not work with 3.7.2.
    - `sudo installer -pkg /path_to/python-3.7.8-macosx10.9.pkg -target /`
- PDM Setup
    - `pdm add tessLATTE`
    - `pdm add certify`
    - `pdm add scikit-learn`
    - `export SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True`
    - `export PATH="/opt/homebrew/opt/openssl@1.1/bin:$PATH"`
- Manual refresh leap seconds file:
```
from astropy.utils import iers
from astropy.utils.data import clear_download_cache

# Clear the download cache
clear_download_cache()

# Update the IERS table
iers.IERS_Auto.open()
```
- Follow the [README](https://github.com/noraeisner/LATTE/tree/master?tab=readme-ov-file) to fix the SSL problem.)
- After running `python -m LATTE`, the window may show up in a separate workspace other than the launching terminal, so flip around and see where it is. Or find the corresponding python launcher icon in the app tray.

## lightkurve Essential API
- `import lightkurve as lk`
### TPF
- `search_result = lk.search_targetpixelfile("Kepler-8", author="Kepler", cadence="long")`
- `tpf = search_result[4].download()`
- `tpf.plot()`
- `tpf.filename`
- `tpf.get_header(ext=0)`
- `tpf.show_properties()`
- `tpf.wcs.pixel_to_world(0, 0)`
- `tpf.time`
- `tpf.flux`, `tpf.flux_err`, `tpf.flux_bkg`, `tpf.flux_bkg_err`
- Alternatively, `tpf.hdu[1].data['FLUX'].shape`
- `lc = tpf.to_lightcurve(aperture_mask=some_mask)`
- `bkg = tpf.get_bkg_lightcurve()`
### Cadence
- `first_cadence = tpf[0]`
- `first_cadence.plot(column='flux')`
- `first_cadence.pipeline_mask`
- `first_cadence.plot(aperture_mask=first_cadence.pipeline_mask)`
### Light Curve
- `search_result = lk.search_lightcurve("Kepler-8", author="Kepler", cadence="long")`
- `lc = search_result[4].download()`
- `lc.plot()`
- `lc.filename`
- `lc.meta`
- `lc.meta["QUARTER"]` or `lc.quarter`
- `clean_lc = lc.remove_outliers(sigma=6)`
### Debug
- `tpf.interact(notebook_url='localhost:8893')`
- `tpf.interact_sky(notebook_url='localhost:8893')`
- `# Only show the first 1000 cadences to makes the tool faster`
- `tpf[0:1000].interact()`
- `# Override the maximum cadence limit`
- `tpf.interact(max_cadences=300000)`
### Aperture
- `am = tpf_crowded.create_threshold_mask(threshold=1)`
- `am.sum()` - counting the number of pixels in the mask.
- Aperture is a 2-D array. The direction of the image y-axis is reverse in the array wrt the image.

## Cotrending Basis Vectors (CBVs)
- Types
    - Single-scale
    - Multi-scale
    - Spike
- Kepler/K2 only has Single-scale.
- TESS has all three.

## Surveys
#### TESS
- Discovery exoplanet transits.
- As of 6 July 2024, TESS had identified 7,203 candidate exoplanets, of which 482 had been confirmed.
- As of July 2024, TESS is undergoing it's 2nd extended mission following the primary and the first extended mission. The 2nd mission ends in 2025.
#### Gaia
- Create 3D map of the Milky Way

## Fix SSLCertverificationError while loading CBVCorrector
```
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context
```